In [ ]:
pip uninstall statsmodels

Found existing installation: statsmodels 0.13.2
Uninstalling statsmodels-0.13.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/statsmodels-0.13.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/statsmodels/*
Proceed (y/n)? Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/uninstall.py", line 86, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_install.py", line 658, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstall.py", line 380, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstall.py", line 423, in _allow

In [ ]:
pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, rcParams, style
style.use('seaborn-darkgrid')
import seaborn as sns
sns.set_style('darkgrid')
from plotly import express as px, graph_objects as go

from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor

import gc
gc.enable()
from warnings import filterwarnings, simplefilter
filterwarnings('ignore')
simplefilter('ignore')

In [ ]:
# 讀黨
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projectB/train.csv',
                    parse_dates = ['date'], infer_datetime_format = True,
                    dtype = {'store_nbr' : 'category',
                             'family' : 'category'},
                    usecols = ['date', 'store_nbr', 'family', 'sales'])
train['date'] = train.date.dt.to_period('D')
train = train.set_index(['date', 'store_nbr', 'family']).sort_index()
train

sales
date       store_nbr family                              
2013-01-01 1         AUTOMOTIVE                     0.000
                     BABY CARE                      0.000
                     BEAUTY                         0.000
                     BEVERAGES                      0.000
                     BOOKS                          0.000
...                                                   ...
2017-08-15 9         POULTRY                      438.133
                     PREPARED FOODS               154.553
                     PRODUCE                     2419.729
                     SCHOOL AND OFFICE SUPPLIES   121.000
                     SEAFOOD                       16.000

[3000888 rows x 1 columns]

In [ ]:
calendar = pd.DataFrame(index = pd.date_range('2013-01-01', '2017-08-31')).to_period('D')
oil = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projectB/oil.csv',
                  parse_dates = ['date'], infer_datetime_format = True,
                  index_col = 'date').to_period('D')
oil['avg_oil'] = oil['dcoilwtico'].rolling(7).mean()
calendar = calendar.join(oil.avg_oil)
calendar['avg_oil'].fillna(method = 'ffill', inplace = True)
calendar.dropna(inplace = True)

In [ ]:
oil

,dcoilwtico,avg_oil
date,,
2013-01-01,NaN,NaN
2013-01-02,93.14,NaN
2013-01-03,92.97,NaN
2013-01-04,93.12,NaN
2013-01-07,93.20,NaN
...,...,...
2017-08-25,47.65,47.720000
2017-08-28,46.40,47.624286
2017-08-29,46.46,47.320000


In [ ]:
n_lags = 3
for l in range(1, n_lags + 1) :
    calendar[f'oil_lags{l}'] = calendar.avg_oil.shift(l)
calendar.dropna(inplace = True)

In [ ]:
holidays = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projectB/holidays_events.csv',
                  parse_dates = ['date'], infer_datetime_format = True,
                  index_col = 'date').to_period('D')
holidays = holidays[holidays.locale == 'National'] # I'm only taking National holiday so there's no false positive.
holidays = holidays.groupby(holidays.index).first() # Removing duplicated holiday at the same date
holidays

,type,locale,locale_name,description,transferred
date,,,,,
2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False
...,...,...,...,...,...
2017-12-22,Additional,National,Ecuador,Navidad-3,False
2017-12-23,Additional,National,Ecuador,Navidad-2,False
2017-12-24,Additional,National,Ecuador,Navidad-1,False


In [ ]:
calendar = calendar.join(holidays) # Joining calendar with holiday dataset
calendar['dofw'] = calendar.index.dayofweek # Weekly day
calendar['wd'] = 1
calendar.loc[calendar.dofw > 4, 'wd'] = 0 # If it's saturday or sunday then it's not Weekday
calendar.loc[calendar.type == 'Work Day', 'wd'] = 1 # If it's Work Day event then it's a workday
calendar.loc[calendar.type == 'Transfer', 'wd'] = 0 # If it's Transfer event then it's not a work day
calendar.loc[calendar.type == 'Bridge', 'wd'] = 0 # If it's Bridge event then it's not a work day
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = 0 # If it's holiday and the holiday is not transferred then it's holiday
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True), 'wd'] = 1 # If it's holiday and transferred then it's not holiday
calendar = pd.get_dummies(calendar, columns = ['dofw'], drop_first = True) # One-hot encoding (Make sure to drop one of the columns by 'drop_first = True')
calendar = pd.get_dummies(calendar, columns = ['type']) # One-hot encoding for type holiday (No need to drop one of the columns because there's a "No holiday" already)
calendar.drop(['locale', 'locale_name', 'description', 'transferred'], axis = 1, inplace = True) # Unused columns
calendar

,avg_oil,oil_lags1,oil_lags2,oil_lags3,wd,dofw_1,dofw_2,dofw_3,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day
2013-01-13,93.284286,93.284286,93.284286,93.218571,0,0,0,0,0,0,1,0,0,0,0,0,0
2013-01-14,93.470000,93.284286,93.284286,93.284286,1,0,0,0,0,0,0,0,0,0,0,0,0
2013-01-15,93.490000,93.470000,93.284286,93.284286,1,1,0,0,0,0,0,0,0,0,0,0,0
2013-01-16,93.644286,93.490000,93.470000,93.284286,1,0,1,0,0,0,0,0,0,0,0,0,0
2013-01-17,93.970000,93.644286,93.490000,93.470000,1,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-27,47.720000,47.720000,47.720000,47.598571,0,0,0,0,0,0,1,0,0,0,0,0,0
2017-08-28,47.624286,47.720000,47.720000,47.720000,1,0,0,0,0,0,0,0,0,0,0,0,0
2017-08-29,47.320000,47.624286,47.720000,47.720000,1,1,0,0,0,0,0,0,0,0,0,0,0
2017-08-30,47.115714,47.320000,47.624286,47.720000,1,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
transcation = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projectB/transactions.csv',
                  parse_dates = ['date'], infer_datetime_format = True,
                  index_col = 'date').to_period('D')
transcation

,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922
...,...,...
2017-08-15,50,2804
2017-08-15,51,1573
2017-08-15,52,2255


In [ ]:
sdate = '2017-04-30' # Start and end of training date
edate = '2017-08-15'

In [ ]:
# sdate = '2017-04-30' # Start and end of training date
# edate = '2017-08-15'

school_season = [] # Feature for school fluctuations
for i, r in calendar.iterrows() :
    if i.month in [4, 5, 8, 9] :
        school_season.append(1)
    else :
        school_season.append(0)
calendar['school_season'] = school_season
calendar

,avg_oil,oil_lags1,oil_lags2,oil_lags3,wd,dofw_1,dofw_2,dofw_3,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,school_season
2013-01-13,93.284286,93.284286,93.284286,93.218571,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2013-01-14,93.470000,93.284286,93.284286,93.284286,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2013-01-15,93.490000,93.470000,93.284286,93.284286,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2013-01-16,93.644286,93.490000,93.470000,93.284286,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2013-01-17,93.970000,93.644286,93.490000,93.470000,1,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-27,47.720000,47.720000,47.720000,47.598571,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2017-08-28,47.624286,47.720000,47.720000,47.720000,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2017-08-29,47.320000,47.624286,47.720000,47.720000,1,1,0,0,0,0,0,0,0,0,0,0,0,1
2017-08-30,47.115714,47.320000,47.624286,47.720000,1,0,1,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
y = train.unstack(['store_nbr', 'family']).loc[sdate:edate]
y

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-04-30        3.0       0.0    0.0     995.0   1.0    139.50700   
2017-05-01        0.0       0.0    2.0     825.0   0.0    116.33900   
2017-05-02        2.0       0.0    2.0    3179.0   0.0    447.23800   
2017-05-03        5.0       0.0    6.0    2479.0   1.0    434.02900   
2017-05-04        3.0       0.0    1.0    2454.0   0.0    438.21400   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0    145.60700   
2017-08-12        6.0       0.0    3.0    1659.0   0.0    243.22000   
2017-08-13        1.0       0.0    1.0     803.0   0.0    136.67900   
2017-08-14        1.0       0.0    6.0    2201.0   0.0    346.03800   
2017-08-15        4.0       0.0    4.0    1942.0   0.0    329.54102   

                                                 ...                     \
store_nbr                                        ...         9            
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES    MEATS   
date                                             ...                      
2017-04-30         2.0    208.0  315.0   60.114  ...       5.0  415.572   
2017-05-01         2.0    227.0  326.0   52.673  ...       5.0  603.395   
2017-05-02        20.0   1061.0  897.0  172.269  ...       0.0  495.275   
2017-05-03        22.0   1117.0  927.0  165.995  ...       4.0  386.662   
2017-05-04        15.0    956.0  755.0  133.780  ...       1.0  601.754   
...                ...      ...    ...      ...  ...       ...      ...   
2017-08-11         4.0    341.0  343.0   64.302  ...       5.0  309.245   
2017-08-12         3.0    351.0  526.0   99.488  ...       2.0  260.298   
2017-08-13         1.0    169.0  266.0   47.770  ...       3.0  327.206   
2017-08-14         4.0    571.0  699.0  154.578  ...      12.0  330.975   
2017-08-15        21.0    703.0  602.0  116.402  ...      11.0  449.228   

                                                                        \
store_nbr                                                                
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS  POULTRY   
date                                                                     
2017-04-30         678.0         10.0                    16.0  513.866   
2017-05-01         950.0         19.0                    18.0  615.898   
2017-05-02         744.0         10.0                    13.0  346.344   
2017-05-03         513.0          5.0                    11.0  432.579   
2017-05-04         487.0          4.0                    13.0  312.411   
...                  ...          ...                     ...      ...   
2017-08-11         373.0         11.0                     2.0  525.224   
2017-08-12         400.0          7.0                    10.0  383.387   
2017-08-13         510.0          2.0                     9.0  412.458   
2017-08-14         445.0          2.0                    14.0  283.429   
2017-08-15         522.0          6.0                     6.0  438.133   

                                                                           
store_nbr                                                                  
family     PREPARED FOODS   PRODUCE SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                                                       
2017-04-30      118.58800  1762.442                        1.0  21.522000  
2017-05-01      175.99100  2502.085                        1.0  54.162000  
2017-05-02      105.04600  2739.654                        5.0  21.362000  
2017-05-03       88.38400  1442.451                        4.0  16.125000  
2017-05-04      104.67000  1285.772                        4.0  11.476000  
...                   ...    

In [ ]:
fourier = CalendarFourier(freq = 'W', order = 4)
dp = DeterministicProcess(index = y.index,
                          order = 1,
                          seasonal = False,
                          constant = False,
                          additional_terms = [fourier],
                          drop = True)
x = dp.in_sample()
x = x.join(calendar)
x

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)",avg_oil,oil_lags1,oil_lags2,...,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,school_season
date,,,,,,,,,,,,,,,,,,,,,
2017-04-30,1.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969,49.358571,49.358571,49.358571,...,0,0,1,0,0,0,0,0,0,1
2017-05-01,2.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,49.154286,49.358571,49.358571,...,0,0,0,0,0,0,1,0,0,1
2017-05-02,3.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969,48.870000,49.154286,49.358571,...,0,0,0,0,0,0,0,0,0,1
2017-05-03,4.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490,48.711429,48.870000,49.154286,...,0,0,0,0,0,0,0,0,0,1
2017-05-04,5.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521,48.187143,48.711429,48.870000,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,104.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521,49.140000,49.252857,49.345714,...,1,0,0,0,0,0,0,1,0,1
2017-08-12,105.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490,49.140000,49.140000,49.252857,...,0,1,0,0,0,0,0,0,0,1
2017-08-13,106.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969,49.140000,49.140000,49.140000,...,0,0,1,0,0,0,0,0,0,1


In [ ]:
y.index

PeriodIndex(['2017-04-30', '2017-05-01', '2017-05-02', '2017-05-03',
             '2017-05-04', '2017-05-05', '2017-05-06', '2017-05-07',
             '2017-05-08', '2017-05-09',
             ...
             '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
             '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
             '2017-08-14', '2017-08-15'],
            dtype='period[D]', name='date', length=108)

In [ ]:
print(y.isna().sum().sum())
display(y)

0


sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-04-30        3.0       0.0    0.0     995.0   1.0    139.50700   
2017-05-01        0.0       0.0    2.0     825.0   0.0    116.33900   
2017-05-02        2.0       0.0    2.0    3179.0   0.0    447.23800   
2017-05-03        5.0       0.0    6.0    2479.0   1.0    434.02900   
2017-05-04        3.0       0.0    1.0    2454.0   0.0    438.21400   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0    145.60700   
2017-08-12        6.0       0.0    3.0    1659.0   0.0    243.22000   
2017-08-13        1.0       0.0    1.0     803.0   0.0    136.67900   
2017-08-14        1.0       0.0    6.0    2201.0   0.0    346.03800   
2017-08-15        4.0       0.0    4.0    1942.0   0.0    329.54102   

                                                 ...                     \
store_nbr                                        ...         9            
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES    MEATS   
date                                             ...                      
2017-04-30         2.0    208.0  315.0   60.114  ...       5.0  415.572   
2017-05-01         2.0    227.0  326.0   52.673  ...       5.0  603.395   
2017-05-02        20.0   1061.0  897.0  172.269  ...       0.0  495.275   
2017-05-03        22.0   1117.0  927.0  165.995  ...       4.0  386.662   
2017-05-04        15.0    956.0  755.0  133.780  ...       1.0  601.754   
...                ...      ...    ...      ...  ...       ...      ...   
2017-08-11         4.0    341.0  343.0   64.302  ...       5.0  309.245   
2017-08-12         3.0    351.0  526.0   99.488  ...       2.0  260.298   
2017-08-13         1.0    169.0  266.0   47.770  ...       3.0  327.206   
2017-08-14         4.0    571.0  699.0  154.578  ...      12.0  330.975   
2017-08-15        21.0    703.0  602.0  116.402  ...      11.0  449.228   

                                                                        \
store_nbr                                                                
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS  POULTRY   
date                                                                     
2017-04-30         678.0         10.0                    16.0  513.866   
2017-05-01         950.0         19.0                    18.0  615.898   
2017-05-02         744.0         10.0                    13.0  346.344   
2017-05-03         513.0          5.0                    11.0  432.579   
2017-05-04         487.0          4.0                    13.0  312.411   
...                  ...          ...                     ...      ...   
2017-08-11         373.0         11.0                     2.0  525.224   
2017-08-12         400.0          7.0                    10.0  383.387   
2017-08-13         510.0          2.0                     9.0  412.458   
2017-08-14         445.0          2.0                    14.0  283.429   
2017-08-15         522.0          6.0                     6.0  438.133   

                                                                           
store_nbr                                                                  
family     PREPARED FOODS   PRODUCE SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                                                       
2017-04-30      118.58800  1762.442                        1.0  21.522000  
2017-05-01      175.99100  2502.085                        1.0  54.162000  
2017-05-02      105.04600  2739.654                        5.0  21.362000  
2017-05-03       88.38400  1442.451                        4.0  16.125000  
2017-05-04      104.67000  1285.772                        4.0  11.476000  
...                   ...    

In [ ]:
xtest = dp.out_of_sample(steps = 16) # 16 because we are predicting next 16 days
xtest = xtest.join(calendar)
xtest

,trend,"sin(1,freq=W-SUN)","cos(1,freq=W-SUN)","sin(2,freq=W-SUN)","cos(2,freq=W-SUN)","sin(3,freq=W-SUN)","cos(3,freq=W-SUN)",avg_oil,oil_lags1,oil_lags2,...,dofw_4,dofw_5,dofw_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day,school_season
2017-08-16,109.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490,48.281429,48.648571,48.934286,...,0,0,0,0,0,0,0,0,0,1
2017-08-17,110.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521,47.995714,48.281429,48.648571,...,0,0,0,0,0,0,0,0,0,1
2017-08-18,111.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521,47.852857,47.995714,48.281429,...,1,0,0,0,0,0,0,0,0,1
2017-08-19,112.0,-0.974928,-0.222521,0.433884,-0.900969,0.781831,0.623490,47.852857,47.852857,47.995714,...,0,1,0,0,0,0,0,0,0,1
2017-08-20,113.0,-0.781831,0.623490,-0.974928,-0.222521,-0.433884,-0.900969,47.852857,47.852857,47.852857,...,0,0,1,0,0,0,0,0,0,1
2017-08-21,114.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,47.688571,47.852857,47.852857,...,0,0,0,0,0,0,0,0,0,1
2017-08-22,115.0,0.781831,0.623490,0.974928,-0.222521,0.433884,-0.900969,47.522857,47.688571,47.852857,...,0,0,0,0,0,0,0,0,0,1
2017-08-23,116.0,0.974928,-0.222521,-0.433884,-0.900969,-0.781831,0.623490,47.645714,47.522857,47.688571,...,0,0,0,0,0,0,0,0,0,1
2017-08-24,117.0,0.433884,-0.900969,-0.781831,0.623490,0.974928,-0.222521,47.598571,47.645714,47.522857,...,0,0,0,0,0,0,0,0,0,1
2017-08-25,118.0,-0.433884,-0.900969,0.781831,0.623490,-0.974928,-0.222521,47.720000,47.598571,47.645714,...,1,0,0,0,0,0,0,0,0,1


In [ ]:
from joblib import Parallel, delayed
import warnings

# Import necessary library
from sklearn.linear_model import Ridge, LinearRegression, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor 
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor

# SEED for reproducible result
SEED = 5

class CustomRegressor():
    
    def __init__(self, n_jobs=-1, verbose=0):
        
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        self.estimators_ = None
        
    def _estimator_(self, X, y):
    
        warnings.simplefilter(action='ignore', category=FutureWarning)
        
        if y.name[2] == 'SCHOOL AND OFFICE SUPPLIES': # Because SCHOOL AND OFFICE SUPPLIES has weird trend, we use decision tree instead.
            r1 = ExtraTreesRegressor(n_estimators = 225, n_jobs=-1, random_state=SEED)
            r2 = RandomForestRegressor(n_estimators = 225, n_jobs=-1, random_state=SEED)
            r3 = XGBRegressor(n_estimators = 225, n_jobs=-1)

            b1 = BaggingRegressor(base_estimator=r1,
                                  n_estimators=10,
                                  n_jobs=-1,
                                  random_state=SEED)
            b2 = BaggingRegressor(base_estimator=r2,
                                  n_estimators=10,
                                  n_jobs=-1,
                                  random_state=SEED)
            
            # b3 = BaggingRegressor(base_estimator=r3,
            #                       n_estimators=10,
            #                       n_jobs=-1,
            #                       random_state=SEED)


            model = VotingRegressor([('et', b1), ('rf', b2)]) # Averaging the result
        else:
            ridge = Ridge(fit_intercept=True, solver='auto', alpha=0.75, normalize=True, random_state=SEED)
            svr = SVR(C = 0.2, kernel = 'rbf')
            xgb = XGBRegressor(n_estimators = 225, n_jobs=-1, random_state=SEED)
            
            model = VotingRegressor([('ridge', ridge), ('svr', svr), ('xxgb', xgb)]) # Averaging result
        model.fit(X, y)

        return model

    def fit(self, X, y):
        from tqdm.auto import tqdm
        
        
        if self.verbose == 0 :
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        else :
            print('Fit Progress')
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in tqdm(range(y.shape[1])))
        return
    
    def predict(self, X):
        from tqdm.auto import tqdm
        if self.verbose == 0 :
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in self.estimators_)
        else :
            print('Predict Progress')
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in tqdm(self.estimators_))
        
        return np.stack(y_pred, axis=1)

In [ ]:
%%time

model = CustomRegressor(n_jobs=-1, verbose=1)
model.fit(x, y)
y_pred = pd.DataFrame(model.predict(x), index=x.index, columns=y.columns)

Fit Progress


  0%|          | 0/1782 [00:00<?, ?it/s]

Predict Progress


  0%|          | 0/1782 [00:00<?, ?it/s]

CPU times: user 3min, sys: 10.2 s, total: 3min 11s
Wall time: 9min 24s


In [ ]:
# from sklearn.metrics import mean_squared_log_error
# y_pred = y_pred.stack(['store_nbr', 'family']).clip(0.)

In [ ]:
ypred = pd.DataFrame(model.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)
ypred

In [ ]:
ypred = ypred.stack(['store_nbr', 'family'])
ypred

In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projectB/sample_submission.csv')
sub['sales'] = ypred.values
sub.to_csv('mytestsubmission2.csv', index = False) # Submit
sub